In [1]:
import numpy as np

In [2]:
class MatrixNorms:
    def __init__(self, matrix):
        self.A = np.array(matrix, dtype=float)

    def norm1(self):
        """Maximum column sum"""
        max_sum = 0
        for j in range(self.A.shape[1]):
            col_sum = sum(abs(self.A[i][j]) for i in range(self.A.shape[0]))
            if col_sum > max_sum:
                max_sum = col_sum
        return max_sum

    def norm_inf(self):
        """Maximum row sum"""
        max_sum = 0
        for i in range(self.A.shape[0]):
            row_sum = sum(abs(x) for x in self.A[i])
            if row_sum > max_sum:
                max_sum = row_sum
        return max_sum

    def frobenius(self):
        """Square root of sum of squares of all elements"""
        total = 0
        for i in range(self.A.shape[0]):
            for j in range(self.A.shape[1]):
                total += self.A[i][j] ** 2
        return total ** 0.5

    def condition_number(self):
        """cond(A) = ||A|| * ||A_inv|| using 1-norm"""
        try:
            A_inv = np.linalg.inv(self.A)
            norm_A = self.norm1()
            norm_inv = MatrixNorms(A_inv).norm1()
            return norm_A * norm_inv
        except np.linalg.LinAlgError:
            return "Matrix is singular (no inverse)"



# User Input Section
def take_input():
    print(" MATRIX NORMS AND CONDITION NUMBER  \n")

    rows = int(input("Enter number of rows: "))
    cols = int(input("Enter number of columns: "))

    matrix = []
    print(f"\nEnter the matrix elements row by row ({cols} numbers per row):")
    for i in range(rows):
        while True:
            row = input(f"Row {i+1}: ").strip().split()
            if len(row) != cols:
                print(f" Please enter exactly {cols} numbers.")
                continue
            try:
                matrix.append([float(x) for x in row])
                break
            except ValueError:
                print(" Please enter valid numeric values.")


# Processing and Output
def display_result_and_verify():
    obj = MatrixNorms(matrix)

    print("\nMatrix A =\n", np.array(matrix))
    print("\n--- Results ---")
    print(f"1-Norm (max column sum):        {obj.norm1():.4f}")
    print(f"Infinity Norm (max row sum):    {obj.norm_inf():.4f}")
    print(f"Frobenius Norm:                 {obj.frobenius():.4f}")
    print(f"Condition Number (1-norm):      {obj.condition_number()}")

In [3]:
class LDUDecomposition:
    def __init__(self, matrix):
        self.A = np.array(matrix, dtype=float)
    """Single-responsibility LDU factorization with unit L and Uhat."""
    def LDU(self,A):
        """
        Input:
            A : (n, n) array_like
        Output:
            L, D, Uhat  such that  A = L @ D @ Uhat
            L    : unit lower-triangular
            D    : diagonal
            Uhat : unit upper-triangular
        """
        
    # Work on a float copy of A
        U = np.array(self.A, dtype=float, copy=True)
        n = U.shape[0]

        if np.isclose(np.linalg.det(A), 0.0):
            raise ValueError("LDU decomposition does not exist for singular matrices (det(A) = 0).")
        
    # Create L as identity
        L = np.eye(n, dtype=float)

    # ----- Forward elimination -----
        for i in range(n):
            pivot = U[i, i]
            if pivot == 0.0:
                raise ZeroDivisionError(f"Zero pivot encountered at i={i}. Cannot proceed.")
            for j in range(i + 1, n):
            # Store multiplier in L
                L[j, i] = U[j, i] / pivot
            # Eliminate below pivot
                U[j, :] -= L[j, i] * U[i, :]

    # Extract D (diagonal part) and make Uhat unit upper
        D = np.eye(n, dtype=float)
        Uhat = U.copy()
        for k in range(n):
            D[k, k] = U[k, k]
            if D[k, k] == 0.0:
                raise ZeroDivisionError(f"Zero diagonal in U at k={k}; cannot form Uhat.")
            Uhat[k, :] /= D[k, k]

        return L, D, Uhat

In [4]:
class GramSchmidt:
    def __init__(self):
        pass

    def find_orthonormal_vectors(self, vectors):
        """
        Given a list of vectors, return an orthonormal set.
        """
        # Convert to float array for accuracy
        A = np.array(vectors, dtype=float)
        num_vectors = len(A)
        dim = len(A[0])

        # List to store orthonormal vectors
        orthonormal_set = []

        for i in range(num_vectors):
            # Start with current vector
            v = A[i]

            # Subtract projections on previous orthonormal vectors
            for u in orthonormal_set:
                proj = np.dot(v, u) * u
                v = v - proj

            # Normalize the new vector
            norm = np.linalg.norm(v)
            if norm > 1e-10:
                u = v / norm
                orthonormal_set.append(u)

        return orthonormal_set




In [5]:

class OrthonormalSet:
    """Facade over GramSchmidt to compute an orthonormal set from input vectors (SRP)."""
    def __init__(self):
        self._gs = GramSchmidt()

    def orthonormalize(self, vectors):
        return self._gs.orthonormalize(vectors)


In [6]:
class QRDecomposition:
    def __init__(self, A):
        self.A = np.array(A, dtype=float)

    def vector_norm(self, v):
        #"Compute Euclidean norm"
        total = 0
        for x in v:
            total += x ** 2
        return total ** 0.5

    def decompose(self):
       #"Perform QR decomposition using Gram-Schmidt process"
        m = len(self.A)          # number of rows
        n = len(self.A[0])       # number of columns

        Q = np.zeros((m, n))
        R = np.zeros((n, n))

        for j in range(n):
            v = self.A[:, j].copy()  # take jth column of A
            for i in range(j):
                R[i][j] = np.dot(Q[:, i], self.A[:, j])
                v = v - R[i][j] * Q[:, i]
            R[j][j] = self.vector_norm(v)
            if R[j][j] != 0:
                Q[:, j] = v / R[j][j]
        return Q, R




In [7]:
class ManualSVD:
    def __init__(self, A):
        # Store the given matrix
        self.A = np.array(A, dtype=float)

    def compute_svd(self):
        """
        Function to compute SVD :
        Steps:
        1. Compute A^T * A
        2. Find eigenvalues and eigenvectors of A^T * A
        3. Compute singular values (square roots of eigenvalues)
        4. Compute V and U
        """
        # Step 1: Compute A^T * A
        ATA = self.A.T @ self.A

        # Step 2: Eigen decomposition of A^T * A
        eigenvalues, V = np.linalg.eig(ATA)

        # Step 3: Sort eigenvalues and corresponding eigenvectors
        # (because eigenvalues may not come in descending order)
        idx = np.argsort(eigenvalues)[::-1]  # indices for sorting
        eigenvalues = eigenvalues[idx]
        V = V[:, idx]

        # Step 4: Compute singular values (σ = sqrt of eigenvalues)
        singular_values = np.sqrt(np.abs(eigenvalues))

        # Step 5: Compute U using U = (1/σ) * A * v_i
        U = []
        for i in range(len(singular_values)):
            if singular_values[i] > 1e-10:  # avoid divide by zero
                ui = (self.A @ V[:, i]) / singular_values[i]
                U.append(ui)
        U = np.array(U).T  # convert to matrix form (columns are u_i)

        # Step 6: Create Σ matrix
        Sigma = np.zeros_like(self.A, dtype=float)
        for i in range(min(len(singular_values), Sigma.shape[0], Sigma.shape[1])):
            Sigma[i, i] = singular_values[i]

        # Return all matrices
        return U, Sigma, V



In [8]:
class LeastSquaresSVD:
    #(Dependency Inversion Principle: depends on numpy only).
    def solve(self, A, b):
        """
        U, Sigma, V = SVD(A)
        rk(A) = # nonzero elements of Sigma
        C = U^T b
        y = zeros(n)
        for i = 1..rk(A):
            y[i] = C[i] / Sigma[i,i]
        x = V y
        return x
        """
        
        A = np.asarray(A, dtype=float)
        b = np.asarray(b, dtype=float).reshape(-1)
        m, n = A.shape

        # U, Sigma, V = SVD(A)
        obj=ManualSVD(A)
        U, Sigma, V = obj.compute_svd()
        
        # rk(A) = # nonzero elements of Sigma (exact, no tolerance)
        s = np.diag(Sigma)                  # singular values
        r = int(np.sum(s != 0.0))

        # C = U^T b
        C = U.T @ b

        # y = zeros(n); y[i] = C[i]/Sigma[i,i] for i = 0..r-1
        y = np.zeros(n, dtype=float)
        for i in range(r):
            y[i] = C[i] / Sigma[i, i]

        # x = V y
        x = V @ y
        return x


In [9]:
class gauss_jacobi():
    def __init__(self,A, b):
        self.A=A
        self.b=b
        
    def solve( self, x0=None, tolerance=1e-10, max_iter=500):
        """
        Solve Ax = b using the Gauss–Jacobi iterative method.

        Parameters
        ----------
        A : (n, n) array_like
            Coefficient matrix (assumed invertible and preferably diagonally dominant).
        b : (n,) array_like
            Right-hand side vector.
        x0 : (n,) array_like, optional
            Initial guess for the solution (defaults to zeros).
        tolerance : float, optional
            Convergence tolerance (default 1e-10).
        max_iter : int, optional
            Maximum number of iterations (default 500).

        Returns
        -------
        x : (n,) ndarray
        Approximate solution.
        num_iter : int
        Number of iterations performed.

        Raises
        ------
        ValueError
        If a zero diagonal entry is found or the method does not converge.
        """

        A = np.array(self.A, dtype=float)#dtype forces elements to be float
        b = np.array(self.b, dtype=float)
        n = len(self.b)

    # Initial guess
        if x0 is None:
            x = np.zeros_like(self.b)#creates a vector of zeros of same length as b
        else:
            x = np.array(x0, dtype=float)

        D = np.diag(self.A) #extracts just the diagonal entries of A as a 1D array
        R = self.A - np.diagflat(D) #creates a 2D diagonal matrix from a 1D array

        for k in range(max_iter):
            x_new = (self.b - np.dot(R, x)) / D #can be seen that this corresponds to the element-wise eqn of G-J
        # Check convergence
            if np.linalg.norm(x_new - x, ord=np.inf) < tolerance:
                return x_new, k 
            x = x_new

        raise ValueError("Jacobi method did not converge within max_iter iterations.")

In [10]:
class GaussSeidel:
    """Iterative Gauss-Seidel solver."""
    def __init__(self,A, b):
        self.A=A
        self.b=b
        
    def solve(self, x0=None, tolerance=1e-10, max_iter=500):
        """
        Solve Ax = b using the Gauss–Seidel iterative method.

        Parameters
        ----------
        A : (n, n) array_like
            Coefficient matrix (assumed invertible and preferably diagonally dominant).
        b : (n,) array_like
            Right-hand side vector.
        x0 : (n,) array_like, optional
            Initial guess for the solution (defaults to zeros).
        tolerance : float, optional
            Convergence tolerance (default 1e-10).
        max_iter : int, optional
            Maximum number of iterations (default 500).

        Returns
        -------
        x : (n,) ndarray
            Approximate solution.
        num_iter : int
            Number of iterations performed.

        Raises
        ------
        ValueError
            If a zero diagonal entry is found or the method does not converge.
        """

        A = np.array(self.A, dtype=float)  # dtype forces elements to be float
        b = np.array(self.b, dtype=float)
        n = len(self.b)

        # Initial guess
        if x0 is None:
            x = np.zeros_like(self.b)  # creates a vector of zeros of same length as b
        else:
            x = np.array(x0, dtype=float)

        D = np.diag(self.A)  # extracts just the diagonal entries of A as a 1D array

        # Gauss–Seidel uses updated values within the same iteration
        for k in range(max_iter):
            x_old = x.copy()  # keep previous iterate to check convergence
            for i in range(n):
                if D[i] == 0:
                    raise ValueError("Zero diagonal entry encountered; division by zero.")
                # split the row sum into parts before and after i
                sum_before = np.dot(A[i, :i], x[:i])          # uses newly updated components
                sum_after  = np.dot(A[i, i+1:], x_old[i+1:])  # uses previous iteration components
                x[i] = (self.b[i] - sum_before - sum_after) / D[i]

            # Check convergence (∞-norm of difference)
            if np.linalg.norm(x - x_old, ord=np.inf) < tolerance:
                return x, k

        raise ValueError("Gauss–Seidel method did not converge within max_iter iterations.")



In [11]:
class ConjugateGradient:
#Conjugate Gradient solver for SPD matrices 
#LSP: can be replaced by any SPD solver
    def __init__(self,A, b):
        self.A=A
        self.b=b
        
    def solve(self, x0=None, eps=1e-10, max_iter=None):
        """
        Solve A x = b using the Conjugate Gradient method.

        Parameters
        ----------
        A : (n, n) array_like
            Coefficient matrix (assumed positive definite).
        b : (n,) array_like
            Right-hand side vector.
        x0 : (n,) array_like, optional
            Initial guess for the solution (defaults to zeros).
        eps : float, optional
            Convergence tolerance. Iteration stops when ||r_{k+1}|| <= eps * ||r_0||.
        max_iter : int, optional
            Maximum number of iterations (default: size of the system).

        Returns
        -------
        x : (n,) ndarray
            Approximate solution vector.
        num_iter : int
            Number of iterations performed.

        Raises
        ------
        ValueError
            If a breakdown occurs (p^T A p <= 0) or the method does not converge
            within the specified maximum number of iterations.

        Notes
        -----
        This implementation follows the pseudocode:
            1. Initialize x⁰, r⁰ = b - A x⁰, p⁰ = r⁰
            2. For k = 0, 1, 2, ...
                  αₖ = <rᵏ, pᵏ> / <pᵏ, A pᵏ>
                  xᵏ⁺¹ = xᵏ + αₖ pᵏ
                  rᵏ⁺¹ = rᵏ - αₖ A pᵏ
                  If ||rᵏ⁺¹|| ≤ ε ||r⁰||, stop
                  βₖ = - <rᵏ⁺¹, A pᵏ> / <pᵏ, A pᵏ>
                  pᵏ⁺¹ = rᵏ⁺¹ + βₖ pᵏ
        """

        A = np.array(self.A, dtype=float)
        b = np.array(self.b, dtype=float)
        n = b.size

        # Initial guess
        if x0 is None:
            x = np.zeros_like(b)
        else:
            x = np.array(x0, dtype=float)

        # Initial residual and direction
        r = b - np.dot(A, x)
        #Here np.dot dots each row of A with x to get Ax 
        p = r.copy()

        r0_norm = np.linalg.norm(r)
        if r0_norm == 0.0:
            return x, 0

        if max_iter is None:
            max_iter = n

        for k in range(1, max_iter + 1):
            Ap = np.dot(A, p)
            pAp = np.dot(p, Ap)
            if pAp <= 0:
                raise ValueError("Breakdown: p^T A p <= 0 (A may not be PD).")

            # Step size
            alpha = np.dot(r, p) / pAp

            # Update x and r
            x = x + alpha * p
            r_new = r - alpha * Ap

            # Check convergence
            rel_res = np.linalg.norm(r_new) / r0_norm
            if rel_res <= eps:
                return x, k

            # Compute β and update p
            beta = -np.dot(r_new, Ap) / pAp
            p = r_new + beta * p

            # Move to next iteration
            r = r_new

        return x, max_iter
        


In [12]:
print("***********************Numerical Linear Algebra Solver*************************")
print("Select a method to perform:")
print("1. Condition Number")
print("2. LDU Decomposition")
print("3. Orthonormal Set (Gram-Schmidt)")
print("4. QR Decomposition")
print("5. SVD (Singular Value Decomposition)")
print("6. Least Squares Solution (LSS)")
print("7. Gauss–Jacobi Method")
print("8. Gauss–Seidel Method")
print("9. Conjugate Gradient Method")

choice = int(input("\nEnter your choice (1–9): "))

***********************Numerical Linear Algebra Solver*************************
Select a method to perform:
1. Condition Number
2. LDU Decomposition
3. Orthonormal Set (Gram-Schmidt)
4. QR Decomposition
5. SVD (Singular Value Decomposition)
6. Least Squares Solution (LSS)
7. Gauss–Jacobi Method
8. Gauss–Seidel Method
9. Conjugate Gradient Method



Enter your choice (1–9):  2


In [13]:
#choice=1
if choice==1 :
    # User Input Section
    #def take_input():
    print(" MATRIX NORMS AND CONDITION NUMBER  \n")

    rows = int(input("Enter number of rows: "))
    cols = int(input("Enter number of columns: "))

    matrix = []
    print(f"\nEnter the matrix elements row by row ({cols} numbers per row):")
    for i in range(rows):
        while True:
            row = input(f"Row {i+1}: ").strip().split()
            if len(row) != cols:
                print(f" Please enter exactly {cols} numbers.")
                continue
            try:
                matrix.append([float(x) for x in row])
                break
            except ValueError:
                print(" Please enter valid numeric values.")

    obj = MatrixNorms(matrix)
    obj.norm1()
    obj.norm_inf()
    obj.frobenius()
    obj.condition_number()
    print("\nMatrix A =\n", np.array(matrix))
    print("\n--- Results ---")
    print(f"1-Norm (max column sum):        {obj.norm1():.4f}")
    print(f"Infinity Norm (max row sum):    {obj.norm_inf():.4f}")
    print(f"Frobenius Norm:                 {obj.frobenius():.4f}")
    print(f"Condition Number (1-norm):      {obj.condition_number()}")


In [14]:
#choice = 2
if choice == 2: 
    rows = int(input("Enter number of rows: "))
    cols = int(input("Enter number of columns: "))

    matrix = []
    print(f"\nEnter the matrix elements row by row ({cols} numbers per row):")
    for i in range(rows):
        while True:
            row = input(f"Row {i+1}: ").strip().split()
            if len(row) != cols:
                print(f" Please enter exactly {cols} numbers.")
                continue
            try:
                matrix.append([float(x) for x in row])
                break
            except ValueError:
                print(" Please enter valid numeric values.")
    obj=LDUDecomposition(matrix)
    L,D,Uhat=obj.LDU(matrix)
    print("L=",L)
    print("D=",D)
    print("Uhat=",Uhat)

Enter number of rows:  2
Enter number of columns:  2



Enter the matrix elements row by row (2 numbers per row):


Row 1:  1 2
Row 2:  3 4


L= [[1. 0.]
 [3. 1.]]
D= [[ 1.  0.]
 [ 0. -2.]]
Uhat= [[ 1.  2.]
 [-0.  1.]]


In [15]:
#choice = 3
if choice == 3:
    n = int(input("Enter number of vectors: "))
    m = int(input("Enter dimension of each vector: "))

    print("\nEnter each vector: ")   # space separated values
    vectors = []
    for i in range(n):
        v = list(map(float, input(f"Vector {i+1}: ").split()))
        vectors.append(v)

    obj = GramSchmidt()

    print("\nGiven Vectors:")
    print(np.round(vectors, 4))

    # Step 1: Calculate orthonormal vectors
    orthonormal_set = obj.find_orthonormal_vectors(vectors)

    print("\nOrthonormal Set Obtained:")
    for i, v in enumerate(orthonormal_set, 1):
        print(f"u{i} =", np.round(v, 4))

    # Step 2: Verification of Orthonormality
    print("VERIFICATION OF ORTHONORMALITY")

    # Convert list to numpy array for matrix operations
    Q = np.array(orthonormal_set)

    # Compute Q * Q^T which should be an identity matrix
    identity_check = np.dot(Q, Q.T)

    print("\nMatrix (Q * Q^T) =")
    print(np.round(identity_check, 4))

    # Check if it's close to an identity matrix
    if np.allclose(identity_check, np.eye(len(Q)), atol=1e-6):
        print("The vectors form an ORTHONORMAL SET.")
    else:
        print("The vectors are NOT perfectly orthonormal (maybe due to rounding errors).")


In [16]:
#choice = 4
if choice == 4:
    print("QR DECOMPOSITION (GRAM–SCHMIDT METHOD) \n")

    rows = int(input("Enter number of rows: "))
    cols = int(input("Enter number of columns: "))

    A = []
    print(f"\nEnter the matrix elements row by row ({cols} numbers per row):")
    for i in range(rows):
        while True:
            row = input(f"Row {i+1}: ").strip().split()
            if len(row) != cols:
                print(f"Please enter exactly {cols} numbers.")
                continue
            try:
                A.append([float(x) for x in row])
                break
            except ValueError:
                print("Please enter valid numeric values.")
    obj=QRDecomposition(A)
    obj.decompose()

    qr = QRDecomposition(A)
    Q, R = qr.decompose()

    print("\nMatrix A =\n", np.array(A))
    print("\nMatrix Q =\n", Q)
    print("\nMatrix R =\n", R)

In [17]:
#choice = 5
if choice == 5:

    rows = int(input("Enter number of rows in the matrix: "))
    cols = int(input("Enter number of columns in the matrix: "))

    print("\nEnter the elements row by row (space separated):")
    A = []
    for i in range(rows):
        row = list(map(float, input(f"Row {i+1}: ").split()))
        A.append(row)

    obj=ManualSVD(A)

    U, Sigma, Vt = obj.compute_svd()

    print("\n--- SINGULAR VALUE DECOMPOSITION RESULT ---")
    print("Given Matrix A =")
    print(np.round(obj.A, 4))

    print("\nMatrix U (Left Singular Vectors) =")
    print(np.round(U, 4))

    print("\nMatrix Σ (Diagonal matrix of singular values) =")
    print(np.round(Sigma, 4))

    print("\nMatrix V^T (Transpose of Right Singular Vectors) =")
    print(np.round(Vt, 4))

    A_reconstructed = U @ Sigma @ Vt
    print("\nReconstructed Matrix (UΣV^T) =")
    print(np.round(A_reconstructed, 4))

    print("\nVerification: Original and reconstructed matrices are almost equal.")
    print("-------------------------------------------------------------")

In [18]:
#choice = 6
if choice == 6:

    m = int(input("Enter number of rows (m): "))
    n = int(input("Enter number of columns (n): "))

    print("Enter elements of matrix A row-wise:")
    A = []
    for i in range(m):
        row = list(map(float, input(f"Row {i+1}: ").split()))
        A.append(row)
    A = np.array(A)

    print("Enter elements of vector b (size m):")
    b = np.array(list(map(float, input().split())))

    obj=LeastSquaresSVD()
    x=obj.solve(A, b)
    
    print("\nSolution vector x:")
    print(x)

    r = np.matmul(A, x) - b
    print("\nResidual vector (A x - b):")
    print(r)

    print("\n2-norm of residual ||A x - b||₂ =", np.linalg.norm(np.matmul(A, x) - b, ord=2))

    # Compare with NumPy's least squares
    x_np, *_ = np.linalg.lstsq(A, b, rcond=None)
    print("\nNumPy lstsq solution for comparison:")
    print(x_np)

    print("\n||x_svd - x_lstsq||₂ =", np.linalg.norm(x - x_np, ord=2))

    # Check normal equations Aᵀ(Ax−b)
    print("\nAᵀ(Ax − b) (should be close to zero):")
    print(np.matmul(A.T, r))

In [19]:
#choice = 7
if choice == 7:
    n = int(input("Enter the number of equations (n): "))

    print("\nEnter the coefficients of matrix A (each row space separated):")
    A = []
    for i in range(n):
        row = list(map(float, input(f"Row {i + 1}: ").split()))
        A.append(row)

    print("\nEnter the constants (right-hand side vector b):")
    b = list(map(float, input().split()))

    obj=gauss_jacobi(A,b)
    x_approx, iterations=obj.solve()


    print("GAUSS–JACOBI METHOD RESULTS")
    print("Coefficient Matrix (A):")
    print(A)
    print("\nRight-Hand Side Vector (b):")
    print(b)

    print(f"\nApproximate Solution (x) after {iterations} iterations:")
    print(np.round(x_approx, 6))

    # Verification using actual inverse method
    try:
        x_actual = np.matmul(np.linalg.inv(A), b)
        print("\nActual Solution (using A⁻¹b):")
        print(np.round(x_actual, 6))

        # Calculate error
        error = np.linalg.norm(x_actual - x_approx)
        print(f"\nVerification Error (||x_actual - x_jacobi||): {error:.6e}")

        if error < 1e-6:
            print(" Verification Successful: Results are accurate.")
        else:
            print("Verification unsuccessful")
    except np.linalg.LinAlgError:
        print("\nMatrix A is singular, so exact solution cannot be computed.")


In [20]:
#choice=8
if choice == 8:
    n = int(input("Enter number of equations (n): "))

    print("\nEnter the coefficients of matrix A (space separated):")
    A = []
    for i in range(n):
        row = list(map(float, input(f"Row {i+1}: ").split()))
        A.append(row)

    print("\nEnter the constants of vector b:")
    b = list(map(float, input().split()))
    
    obj=GaussSeidel(A,b)
    x_approx, iterations=obj.solve()


    print("GAUSS–SEIDEL METHOD RESULTS")
    print("Matrix A:")
    print(A)
    print("\nVector b:")
    print(b)

    print(f"\nApproximate Solution (after {iterations} iterations):")
    print(np.round(x_approx, 6))

    
        # Verify by comparing with actual solution
    try:
        x_actual = np.matmul(np.linalg.inv(A), b)
        print("\nActual Solution (A⁻¹b):")
        print(np.round(x_actual, 6))

        # Compute error
        error = np.linalg.norm(x_actual - x_approx)
        print(f"\nVerification Error (||x_actual - x_GS||): {error:.6e}")

        if error < 1e-6:
            print(" Verification Successful: Results are accurate.")
        else:
            print(" Verification unuccessful.")
    except np.linalg.LinAlgError:
        print("\nMatrix A is singular, cannot compute exact solution.")

In [21]:
#choice=9
if choice == 9:
    n = int(input("Enter number of equations (n): "))

    print("\nEnter the coefficients of matrix A (space separated):")
    A = []
    for i in range(n):
        row = list(map(float, input(f"Row {i+1}: ").split()))
        A.append(row)

    print("\nEnter the constants of vector b:")
    b = list(map(float, input().split()))
    
    obj=ConjugateGradient(A,b)
    x, iterations=obj.solve()

    print("\n--- Conjugate Gradient Method Result ---")
    print("Solution Vector x:")
    for i in range(len(x)):
        print(f"x{i+1} = {x[i]:.6f}")

    print("\nNumber of iterations:", iterations)

    # Verification
    print("\n--- Verification ---")
    print("Computing A * x ...")
    Ax = np.dot(A, x)
    print("A * x =", Ax)
    print("b =", b)
    print("\nError (b - A*x) =", b - Ax)

    # Check if A*x ≈ b
    if np.allclose(Ax, b, atol=1e-6):
        print(" Verification successful: A*x is approximately equal to b.")
    else:
        print(" Verification unsuccessful")